In [ ]:
!pip install -q tensorflow_io tensorflow-text

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_text as tf_text

In [ ]:
train_dataframe = pd.read_csv("../input/hubmap-organ-segmentation/train.csv")
train_dataframe.head()

In [ ]:
TRAIN_IMAGE_PATH = "../input/hubmap-organ-segmentation/train_images"
TRAIN_ANNOTATION_PATH = "../input/hubmap-organ-segmentation/train_annotations"
TEST_IMAGE_PATH = "../input/hubmap-organ-segmentation/test_images"

In [ ]:
train_image_files = [os.path.join(TRAIN_IMAGE_PATH, f"{_id}.tiff") for _id in train_dataframe.id]
train_rles = list(train_dataframe.rle)

In [ ]:
@tf.function
def preprocess(image, mask_rle):
    image = tf.io.read_file(image)
    image = tfio.experimental.image.decode_tiff(image)
    image.set_shape([None, None, 4])
    
    def _preprocess(image, mask_rle):
        """Reference: https://www.kaggle.com/code/paulorzp/run-length-encode-and-decode/script"""
        image = image.numpy()
        mask_rle = mask_rle.numpy()
        s = mask_rle.split()
        shape = (image.shape[-3], image.shape[-2])
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        mask = np.zeros(shape[0] * shape[1], dtype=np.uint8)
        for lo, hi in zip(starts, ends):
            mask[lo:hi] = 1
        mask = mask.reshape(*shape, 1)
        return image, mask
    
    return tf.py_function(_preprocess, [image, mask_rle], [tf.float32, tf.uint8])


train_dataset = tf.data.Dataset.from_tensor_slices((train_image_files, train_rles))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
x, y = next(iter(train_dataset))
x.shape, y.shape

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(x.numpy().astype(np.uint8))
plt.imshow(y, cmap='coolwarm', alpha=0.5)
plt.axis("off")
plt.show()

### Work In Progress